# Formative00 Part 4 — System Check

Run **Run → Run All Cells**.

## What this notebook checks
- Your Python environment is working (imports + versions)
- Optional: your laptop can reach **Local Ollama** (`http://localhost:11434`)
- Required: you can test a **Campus LLM endpoint** (you will paste the base URL)
- Required: you can test **Cloud OpenAI via the local proxy** (`http://localhost:8001/chat`)

If you see errors, screenshot them for your OneNote artifact.


In [ ]:
import sys, platform
print('Python:', sys.version.split()[0])
print('Platform:', platform.platform())


In [ ]:
import pandas as pd
import matplotlib
import requests
print('pandas:', pd.__version__)
print('matplotlib:', matplotlib.__version__)
print('requests:', requests.__version__)


## 1) Local Ollama test (laptop)
This may fail if Ollama is not installed/running yet.


In [ ]:
import requests
try:
    r = requests.get('http://localhost:11434/api/tags', timeout=3)
    print('Ollama reachable:', r.status_code)
    if r.ok:
        data = r.json()
        print('Models found:', [m['name'] for m in data.get('models', [])][:10])
except Exception as e:
    print('Ollama not reachable yet:', type(e).__name__, e)


## 2) Campus endpoint test (required)

Paste the campus base URL in the cell below.
Use the format: `http://10.x.x.x:11434/v1`


In [ ]:
# TODO: Paste the campus base URL provided in class
CAMPUS_BASE_URL = "http://10.0.0.10:11434/v1"  # change this

import requests, json

payload = {
    "model": "llama3.2:1b",
    "messages": [{"role": "user", "content": "Reply with exactly: CAMPUS_OK"}],
    "temperature": 0.0,
}

try:
    r = requests.post(f"{CAMPUS_BASE_URL}/chat/completions", json=payload, timeout=10)
    print('Campus status:', r.status_code)
    data = r.json()
    text = (data.get('choices') or [{}])[0].get('message', {}).get('content', '')
    print('Campus response:', text)
except Exception as e:
    print('Campus test failed:', type(e).__name__, e)


## 3) Cloud OpenAI test via local proxy (required)

This notebook does **not** call OpenAI directly.
It calls your **local proxy** at `http://localhost:8001/chat`.

Before running this cell, you must:
1) Set your key in PowerShell: `$env:OPENAI_API_KEY = "..."`
2) Start the proxy in another terminal: `uvicorn llm_proxy_fastapi:app --port 8001`


In [ ]:
import requests

payload = {
    "provider": "openai",
    "model": "gpt-4o-mini",
    "prompt": "Reply with exactly: CLOUD_OK"
}

try:
    r = requests.post('http://localhost:8001/chat', json=payload, timeout=20)
    print('Proxy status:', r.status_code)
    data = r.json()
    print('Cloud via proxy response:', data.get('text',''))
except Exception as e:
    print('Cloud via proxy test failed:', type(e).__name__, e)


## Final check
If you reached the campus endpoint and the cloud proxy successfully, you are ready for Part 4.


In [ ]:
print('✅ System check completed')
